# SHA1 key fix for the Elasticsearch Signing Key

The Elasticsearch Signing Key can't currently be imported on RHEL9 becuase it has self signatures that use SHA1 as the digest algorithm and RHEL9 no longer recognizes SHA1 as valid. This notebook demonstrates the issue and a solution that can be applied without migrating to a new key.

## Setup

#### Get a RHEL9 machine

In [6]:
!echo -e '\
Vagrant.configure("2") do |config|\n\
  config.vm.box = "generic/rhel9"\n\
  config.vm.synced_folder ".", "/vagrant"\n\
end\n\
' > Vagrantfile

In [7]:
!vagrant up

Bringing machine 'default' up with 'virtualbox' provider...
==> default: Importing base box 'generic/rhel9'...
Progress: 90%==> default: Matching MAC address for NAT networking...
==> default: Checking if box 'generic/rhel9' version '4.2.16' is up to date...
==> default: Setting the name of the VM: sha1-key-fix_default_1691418654752_40751
==> default: Clearing any previously set network interfaces...
==> default: Preparing network interfaces based on configuration...
    default: Adapter 1: nat
==> default: Forwarding ports...
    default: 22 (guest) => 2222 (host) (adapter 1)
==> default: Running 'pre-boot' VM customizations...
==> default: Booting VM...
==> default: Waiting for machine to boot. This may take a few minutes...
    default: SSH address: 127.0.0.1:2222
    default: SSH username: vagrant
    default: SSH auth method: private key
    default: 
    default: Vagrant insecure key detected. Vagrant will automatically replace
    default: this with a newly generated keypair for

## The issue

#### Get the Elasticsearch Signing Key and attempt to import it

In [8]:
!curl -s 'https://artifacts.elastic.co/GPG-KEY-elasticsearch' > GPG-KEY-elasticsearch

In [9]:
!vagrant ssh -c 'sudo rpm --import /vagrant/GPG-KEY-elasticsearch'

error: /vagrant/GPG-KEY-elasticsearch: key 1 import failed.


This is the problem we want to fix.

#### Examine the key details

In [10]:
!gpg --with-fingerprint GPG-KEY-elasticsearch 2>/dev/null

pub   rsa2048/0xD27D666CD88E42B4 2013-09-16 [SC]
      Key fingerprint = 4609 5ACC 8548 582C 1A26  99A9 D27D 666C D88E 42B4
uid                             Elasticsearch (Elasticsearch Signing Key) <dev_ops@elasticsearch.org>
sub   rsa2048/0xAB6B7FCB60D31954 2013-09-16 [E]
      Key fingerprint = 3B0C 6695 3876 82E1 8F77  B489 AB6B 7FCB 60D3 1954


In [11]:
!gpg --list-packets GPG-KEY-elasticsearch

# off=0 ctb=99 tag=6 hlen=3 plen=269
:public key packet:
	version 4, algo 1, created 1379344074, expires 0
	pkey[0]: [2048 bits]
	pkey[1]: [17 bits]
	keyid: D27D666CD88E42B4
# off=272 ctb=b4 tag=13 hlen=2 plen=69
:user ID packet: "Elasticsearch (Elasticsearch Signing Key) <dev_ops@elasticsearch.org>"
# off=343 ctb=89 tag=2 hlen=3 plen=312
:signature packet: algo 1, keyid D27D666CD88E42B4
	version 4, created 1379344074, md5len 0, sigclass 0x13
	digest algo 2, begin of digest 73 8c
	hashed subpkt 2 len 4 (sig created 2013-09-16)
	hashed subpkt 27 len 1 (key flags: 03)
	hashed subpkt 11 len 5 (pref-sym-algos: 9 8 7 3 2)
	hashed subpkt 21 len 5 (pref-hash-algos: 8 2 9 10 11)
	hashed subpkt 22 len 3 (pref-zip-algos: 2 3 1)
	hashed subpkt 30 len 1 (features: 01)
	hashed subpkt 23 len 1 (keyserver preferences: 80)
	subpkt 16 len 8 (issuer key ID D27D666CD88E42B4)
	data: [2048 bits]
# off=658 ctb=b9 tag=14 hlen=3 plen=269
:public sub key packet:
	version 4, algo 1, created 1379344074, expires 

RSA (Encrypt or Sign) is the public key algorithm used, indicated by `algo 1` in the `public key packet`, `signature packet`, and `public sub key packet` sections.

SHA1 is the digest algorithm used for the signatures, indicated by `digest algo 2` in the `signature packet` sections. This needs to change.

## The solution

To demonstrate the solution we'll start with a new key that uses SHA1 as its digest algorithm. Then we'll edit the key to trigger the generation of new self-signatures, verify that the signatures have changed and that the key IDs and fingerprints have not changed, and finally, show that the fixed key can be imported successfully.

#### Generate a key with SHA1 signatures

In [12]:
!echo -e '\
%echo Generating a basic OpenPGP key\n\
Key-Type: RSA\n\
Key-Length: 2048\n\
Subkey-Type: RSA\n\
Subkey-Length: 2048\n\
Name-Real: My Name\n\
Name-Comment: Some Comment\n\
Name-Email: keymaster@example.com\n\
Expire-Date: 0\n\
Passphrase: some-passphrase\n\
%commit\n\
%echo done\
' > key-options.txt

In [13]:
!gpg --options <(echo -e "cert-digest-algo SHA1") --allow-weak-key-signatures --batch --generate-key key-options.txt

gpg: Generating a basic OpenPGP key
gpg: revocation certificate stored as '/home/chrisberkhout/.gnupg/openpgp-revocs.d/E51EB50C358F7FD6AFF778E0503D4EEE85D18576.rev'
gpg: done


In [14]:
!gpg --armor --export keymaster@example.com > publickey-sha1.asc

In [15]:
!gpg --with-fingerprint publickey-sha1.asc 2>/dev/null | tee publickey-sha1-fingerprints.txt

pub   rsa2048/0x503D4EEE85D18576 2023-08-07 [SCEA]
      Key fingerprint = E51E B50C 358F 7FD6 AFF7  78E0 503D 4EEE 85D1 8576
uid                             My Name (Some Comment) <keymaster@example.com>
sub   rsa2048/0xF21B1176DBB44885 2023-08-07 [SEA]
      Key fingerprint = 23F7 B353 929D 7658 5D35  1C12 F21B 1176 DBB4 4885


In [16]:
!gpg --list-packets publickey-sha1.asc | tee publickey-sha1-packets.txt

# off=0 ctb=99 tag=6 hlen=3 plen=269
:public key packet:
	version 4, algo 1, created 1691418803, expires 0
	pkey[0]: [2048 bits]
	pkey[1]: [17 bits]
	keyid: 503D4EEE85D18576
# off=272 ctb=b4 tag=13 hlen=2 plen=46
:user ID packet: "My Name (Some Comment) <keymaster@example.com>"
# off=320 ctb=89 tag=2 hlen=3 plen=334
:signature packet: algo 1, keyid 503D4EEE85D18576
	version 4, created 1691418803, md5len 0, sigclass 0x13
	digest algo 2, begin of digest 34 86
	hashed subpkt 33 len 21 (issuer fpr v4 E51EB50C358F7FD6AFF778E0503D4EEE85D18576)
	hashed subpkt 2 len 4 (sig created 2023-08-07)
	hashed subpkt 27 len 1 (key flags: 2F)
	hashed subpkt 11 len 4 (pref-sym-algos: 9 8 7 2)
	hashed subpkt 21 len 5 (pref-hash-algos: 10 9 8 11 2)
	hashed subpkt 22 len 3 (pref-zip-algos: 2 3 1)
	hashed subpkt 30 len 1 (features: 01)
	hashed subpkt 23 len 1 (keyserver preferences: 80)
	subpkt 16 len 8 (issuer key ID 503D4EEE85D18576)
	data: [2042 bits]
# off=657 ctb=b9 tag=14 hlen=3 plen=269
:public sub key

Notice that each signature packet has `digest algo 2`, indicating that SHA1 is used.

#### Edit to trigger new self signatures

GnuPG won't let you manually add a self signature to a key that already has a self signature, but some changes to the key will trigger it to automatically generate new self signatures.

This is usually an interactive process. It's automated below using `expect`.

The changes are:
- Set updated algorithm preferences (to update the primary key's signature)
- Set an unchanged subkey expiry (to update the subkey's signature)

In [17]:
!echo -e '\
#!/usr/bin/expect -f\n\
set timeout -1\n\
spawn gpg --pinentry-mode=loopback --passphrase "some-passphrase" --edit-key keymaster@example.com\n\
expect "gpg> "\n\
send -- "setpref SHA512 SHA384 SHA256 SHA224 AES256 AES192 AES CAST5 ZLIB BZIP2 ZIP Uncompressed\\r"\n\
expect "Really update the preferences? (y/N) "\n\
send -- "y\\r"\n\
expect "gpg> "\n\
send -- "key 1\r"\n\
expect "gpg> "\n\
send -- "expire\\r"\n\
expect "Key is valid for? (0) "\n\
send -- "0\\r"\n\
expect "Is this correct? (y/N) "\n\
send -- "y\\r"\n\
expect "gpg> "\n\
send -- "save\\r"\n\
expect eof\n\
' | expect -f -

spawn gpg --pinentry-mode=loopback --passphrase some-passphrase --edit-key keymaster@example.com
gpg (GnuPG) 2.2.41; Copyright (C) 2022 g10 Code GmbH
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.

Secret key is available.

gpg: checking the trustdb
gpg: marginals needed: 3  completes needed: 1  trust model: pgp
gpg: depth: 0  valid:   1  signed:   0  trust: 0-, 0q, 0n, 0m, 0f, 1u
sec  rsa2048/0x503D4EEE85D18576
     created: 2023-08-07  expires: never       usage: SCEA
     trust: ultimate      validity: ultimate
ssb  rsa2048/0xF21B1176DBB44885
     created: 2023-08-07  expires: never       usage: SEA 
[ultimate] (1). My Name (Some Comment) <keymaster@example.com>

gpg> setpref SHA512 SHA384 SHA256 SHA224 AES256 AES192 AES CAST5 ZLIB BZIP2 ZIP Uncompressed
Set preference list to:
     Cipher: AES256, AES192, AES, CAST5, 3DES
     AEAD: 
     Digest: SHA512, SHA384, SHA256, SHA224, SHA1
     Compression: ZLIB, BZ

Now we can export and examine the updated key.

In [18]:
!gpg --armor --export keymaster@example.com > publickey-sha512.asc

In [19]:
!gpg --with-fingerprint publickey-sha512.asc 2>/dev/null | tee publickey-sha512-fingerprints.txt

pub   rsa2048/0x503D4EEE85D18576 2023-08-07 [SCEA]
      Key fingerprint = E51E B50C 358F 7FD6 AFF7  78E0 503D 4EEE 85D1 8576
uid                             My Name (Some Comment) <keymaster@example.com>
sub   rsa2048/0xF21B1176DBB44885 2023-08-07 [SEA]
      Key fingerprint = 23F7 B353 929D 7658 5D35  1C12 F21B 1176 DBB4 4885


In [20]:
!gpg --list-packets publickey-sha512.asc | tee publickey-sha512-packets.txt

# off=0 ctb=99 tag=6 hlen=3 plen=269
:public key packet:
	version 4, algo 1, created 1691418803, expires 0
	pkey[0]: [2048 bits]
	pkey[1]: [17 bits]
	keyid: 503D4EEE85D18576
# off=272 ctb=b4 tag=13 hlen=2 plen=46
:user ID packet: "My Name (Some Comment) <keymaster@example.com>"
# off=320 ctb=89 tag=2 hlen=3 plen=334
:signature packet: algo 1, keyid 503D4EEE85D18576
	version 4, created 1691419540, md5len 0, sigclass 0x13
	digest algo 10, begin of digest ec 2c
	hashed subpkt 27 len 1 (key flags: 2F)
	hashed subpkt 30 len 1 (features: 01)
	hashed subpkt 23 len 1 (keyserver preferences: 80)
	hashed subpkt 33 len 21 (issuer fpr v4 E51EB50C358F7FD6AFF778E0503D4EEE85D18576)
	hashed subpkt 2 len 4 (sig created 2023-08-07)
	hashed subpkt 11 len 4 (pref-sym-algos: 9 8 7 3)
	hashed subpkt 21 len 4 (pref-hash-algos: 10 9 8 11)
	hashed subpkt 22 len 4 (pref-zip-algos: 2 3 1 0)
	subpkt 16 len 8 (issuer key ID 503D4EEE85D18576)
	data: [2048 bits]
# off=657 ctb=b9 tag=14 hlen=3 plen=269
:public sub ke

Note that `digest algo 10` (SHA512) is used for each signature. There is mention of `digest algo 2` later in the subkey information, but we will verify that this doesn't prevent successful key import.

#### Verify that the signatures have changed and key IDs and fingerprints have not changed

Let's compare the packet data from the initial version of the key with the updated version:

In [21]:
!colordiff -u99 publickey-sha1-packets.txt publickey-sha512-packets.txt

--- publickey-sha1-packets.txt	2023-08-07 16:33:47.656839137 +0200
+++ publickey-sha512-packets.txt	2023-08-07 16:46:36.120187016 +0200
@@ -1,38 +1,38 @@
 # off=0 ctb=99 tag=6 hlen=3 plen=269
 :public key packet:
 	version 4, algo 1, created 1691418803, expires 0
 	pkey[0]: [2048 bits]
 	pkey[1]: [17 bits]
 	keyid: 503D4EEE85D18576
 # off=272 ctb=b4 tag=13 hlen=2 plen=46
 :user ID packet: "My Name (Some Comment) <keymaster@example.com>"
 # off=320 ctb=89 tag=2 hlen=3 plen=334
 :signature packet: algo 1, keyid 503D4EEE85D18576
-	version 4, created 1691418803, md5len 0, sigclass 0x13
-	digest algo 2, begin of digest 34 86
-	hashed subpkt 33 len 21 (issuer fpr v4 E51EB50C358F7FD6AFF778E0503D4EEE85D18576)
-	hashed subpkt 2 len 4 (sig created 2023-08-07)
+	version 4, created 1691419540, md5len 0, sigclass 0x13
+	digest algo 10, begin of digest ec 2c
 	hashed subpkt 27 len 1 (key flags: 2F)
-	hashed subpkt 11 len 4 (pref-sym-algos: 9 8 7 2)
-	hashed subpkt 21 len 5 (pref-hash-algos: 10 9 8 1

Notice that no key IDs have changed. The `pref-hash-algos` no longer includes `2` (SHA1) and signatures now use `digest aglo 10` (SHA512).

When we compare the fingerprints, there are no changes:

In [23]:
!colordiff -u99 publickey-sha1-fingerprints.txt publickey-sha512-fingerprints.txt | wc -l

0


#### Verify the fixed key imports succesfully

When we attempt to import each version of this key with `rpm`, we see that the SHA1 version fails, and the SHA512 version succeeds:

In [24]:
!vagrant ssh -c 'sudo rpm --import /vagrant/publickey-sha1.asc'

error: /vagrant/publickey-sha1.asc: key 1 import failed.


In [25]:
!vagrant ssh -c 'sudo rpm --import /vagrant/publickey-sha512.asc'

In [26]:
!vagrant ssh -c 'rpm -q --queryformat "%{SUMMARY}\n" gpg-pubkey'

Red Hat, Inc. (release key 2) <security@redhat.com> public key
Red Hat, Inc. (auxiliary key 3) <security@redhat.com> public key
Fedora (epel9) <epel@fedoraproject.org> public key
My Name (Some Comment) <keymaster@example.com> public key


## Clean up

Delete keys the generated key from the local keyring:

In [76]:
!fingerprint=`gpg --fingerprint 'keymaster@example.com' | grep fingerprint | head -n1 | cut -d= -f2 | sed 's/ //g'` &&\
gpg --batch --yes --delete-secret-keys $fingerprint &&\
gpg --batch --yes --delete-keys $fingerprint

gpg: checking the trustdb
gpg: marginals needed: 3  completes needed: 1  trust model: pgp
gpg: depth: 0  valid:   1  signed:   0  trust: 0-, 0q, 0n, 0m, 0f, 1u


In [77]:
!gpg --list-keys 'keymaster@example.com'

gpg: checking the trustdb
gpg: no ultimately trusted keys found
gpg: error reading key: No public key


Suspend and destroy the RHEL9 Vagrant machine:

In [28]:
!vagrant suspend

==> default: Saving VM state and suspending execution...


In [29]:
!vagrant destroy -f

==> default: Discarding saved state of VM...
==> default: Destroying VM and associated drives...


Remove the generated files:

In [31]:
!rm -f \
 Vagrantfile\
 GPG-KEY-elasticsearch\
 key-options.txt\
 publickey-sha1.asc\
 publickey-sha1-fingerprints.txt\
 publickey-sha1-packets.txt\
 publickey-sha512.asc\
 publickey-sha512-fingerprints.txt\
 publickey-sha512-packets.txt